In [195]:
import numpy as np   #importing libraries
import simpy 
import queue
import random
import random as rd
import matplotlib 
from matplotlib import pyplot as plt

In [196]:
global SZ,CP,RM
SZ=[]
CP=[]
RM=[]
N=100
Lambda = 1
mu = 1.25
#t = 0   #system clock
t_lambda = 0 #poisson random variable used to generate next service time
t_mu = 0  #poisson random variable used to generate next departure time
t_a=0
wait=0
R=1000
transmission_time=0
l="0"
P_transmission=500
P_idle=100


In [197]:
class Packet(object):
    def __init__(self, time, id, mobileID, seeed):
        
        self.time=time                                                
        self.id=id
        self.mobileID=mobileID   
        self.size=random.randint(200,400) #seed 1 op 2, seed 2 op 
        SZ.append(round(self.size,2))
        #print(seeed)
        #random.seed(seeed)
        self.cpu=random.randint(20,90) #seed 1 op 2, seed 2 op 
        CP.append(round(self.cpu,2))
        #random.seed(seeed)
        self.ram=random.randint(1, 32) #seed 1 op 3, seed 2 op 1, seed 3 op 4
        RM.append(round(self.ram,2))

class mobile(object): #mobile class with an ID              
    def __init__(self, ID):
        self.ID=ID
        self.pot=int(1)
        
    #generates packets and sends via out_pipe
    def packet_generator(self, numPackets, env, out_pipe, task_id,send): 
        
        yield env.timeout(send)
        print("sending task {} of mobile {} at time {}".format(task_id, self.ID, env.now))
        p=Packet(env.now, task_id, self.ID, self.pot)
        self.pot+=1
        if(self.pot==4):
                self.pot=1
        yield out_pipe.put(p)
        item = yield out_pipe.get()



In [198]:
 def reward_generator(T): 
    Tsla=2.5 
    Nsla=3
    Gsla=3  
    Cost_sla=50
    Cg=0
    Ct=0
    Cn=0
    Cost_n=0
    
    #if (G<Gsla):
        #Cg=-5*(Gsla/G)
    if (T>Tsla):
        Ct = -5 * (T/Tsla)
    #if (N>Nsla):
        #Cn=-5*(N/Nsla)
    #if (cost>Cost_sla):
        #Cost_n=-5*(cost/Cost_sla)
    
    Reward = (Tsla/T) + Ct #Reward Function
    return Reward

In [199]:
class Fog(object):      
    def __init__(self, ID):
        self.ID=ID
        self.Number_queue=0
        self.lost=0
        self.response_time=0
        
        self.service_time=[]
        self.response_time_list=[]
        self.Serverqueue=[]
        self.waitlist=[]
        self.energy_offloading=[]
        self.reward_list=[]
        
    def FOG(self, env, server, in_pipe):
        
         msg = yield in_pipe.get()
         transmission_time = (msg.size/R)
         #print("transmits task {} to need time {}".format(msg.id,transmission_time,self.ID))
         yield env.timeout(transmission_time)
         a=env.now
         #print("Arrived task {} at time {} at FOG {}".format(msg.id,a,self.ID))
         
         u=rd.random()
         t_mu=-np.log(u)/mu
         self.service_time.append(t_mu)
        
         E = (P_transmission * transmission_time) + (P_idle * t_mu)
         self.energy_offloading.append(E)
        
         with server.request() as req:
             yield req
             s=env.now
             wait=s-a
             #print("Starting service task {} at time {} at FOG {}".format(msg.id,s,self.ID))
             #print("Waiting time of task {} at time {} at FOG {}".format(msg.id,wait,self.ID))
             self.waitlist.append(wait)
             self.response_time=wait+t_mu
             self.response_time_list.append(self.response_time)
            
             if wait>0:
                if len(self.Serverqueue)<10:
                    self.Serverqueue.append(msg)
                    self.Number_queue = self.Number_queue + 1
                else:
                    self.lost=self.lost+1 
                    print("Packet lost")
                    return 
             yield env.timeout(t_mu)
             #print("Finish service task {} at time {} at FOG {}".format(msg.id,env.now,self.ID))
             if len(self.Serverqueue)>0:
                 self.Serverqueue.pop(0)
             
             r=reward_generator(self.response_time)
             self.reward_list.append(r)

In [200]:
env1 = simpy.Environment()    
server = simpy.Resource(env1, capacity=3)

pipe1 = simpy.Store(env1)
pipe2 = simpy.Store(env1)
pipe3 = simpy.Store(env1)

ms1=mobile(1)
ms2=mobile(2)
ms3=mobile(3)

fog1 = Fog(1)
fog2 = Fog(2)

for i in range(N):
    v=rd.randint(1,990) 
    t_lambda=v/Lambda

    env1.process(ms1.packet_generator(1, env1, pipe1, i,t_lambda))
    env1.process(fog1.FOG(env1, server, pipe1))
    
    v=rd.randint(1,990)
    t_lambda=v/Lambda

    env1.process(ms2.packet_generator(1, env1, pipe2, i,t_lambda))
    env1.process(fog2.FOG(env1, server, pipe2))
    
env1.run()

#print(fog1.lost,fog2.lost)

#Na=N-fog.lost

print("FOG-1 RESPONSE TIME:", fog1.response_time_list)
print("FOG-1 REAWARD:", fog1.reward_list)
print("FOG-1 MAXIMUM REAWARD:", max(fog1.reward_list))
print("FOG-1 total number of task in queue",fog1.Number_queue)
avgwaittime=sum(fog1.waitlist)/N
print("FOG-1 Average queueing time:", avgwaittime)
avgservicetime=sum(fog1.service_time)/N
print("FOG-1 Average service time:", avgservicetime)
avgsystemtime=sum(fog1.response_time_list)/N
print("FOG-1 Average response time:", avgsystemtime)
avgenergy=sum(fog1.energy_offloading)/N
print("FOG-1 Average Energy consumption:", avgenergy)

print("FOG-2 RESPONE TIME:", fog2.response_time_list)
print("FOG-2 REAWARD:", fog2.reward_list)
print("FOG-2 MAXIMUM REAWARD:", max(fog2.reward_list))
print("FOG-2 total number of task in queue",fog2.Number_queue)
avgwaittime=sum(fog2.waitlist)/N
print("FOG-2 Average queueing time:", avgwaittime)
avgservicetime=sum(fog2.service_time)/N
print("FOG-2 Average service time:", avgservicetime)
avgsystemtime=sum(fog2.response_time_list)/N
print("FOG-2 Average response time:", avgsystemtime)
avgenergy=sum(fog2.energy_offloading)/N
print("FOG-2 Average Energy consumption:", avgenergy)


sending task 31 of mobile 2 at time 3.0
sending task 3 of mobile 1 at time 4.0
sending task 70 of mobile 1 at time 8.0
sending task 96 of mobile 2 at time 26.0
sending task 37 of mobile 1 at time 35.0
sending task 94 of mobile 2 at time 37.0
sending task 56 of mobile 1 at time 50.0
sending task 65 of mobile 2 at time 50.0
sending task 87 of mobile 2 at time 60.0
sending task 60 of mobile 1 at time 67.0
sending task 13 of mobile 2 at time 68.0
sending task 62 of mobile 2 at time 74.0
sending task 21 of mobile 2 at time 87.0
sending task 69 of mobile 2 at time 89.0
sending task 61 of mobile 2 at time 95.0
sending task 28 of mobile 2 at time 96.0
sending task 61 of mobile 1 at time 99.0
sending task 59 of mobile 2 at time 101.0
sending task 79 of mobile 1 at time 119.0
sending task 72 of mobile 2 at time 123.0
sending task 94 of mobile 1 at time 124.0
sending task 25 of mobile 2 at time 141.0
sending task 15 of mobile 1 at time 150.0
sending task 63 of mobile 1 at time 161.0
sending task 